1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchainhub --quiet
%pip install langchain-pinecone

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter = text_splitter)

len(document_list)

In [13]:
from langchain_ollama import OllamaEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()


embedding = OllamaEmbeddings(model=os.getenv("embedmodel"))

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os,time

INDEX = "tax-markdown-index"
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

if INDEX in [i["name"] for i in pc.list_indexes()]:
    pc.delete_index(INDEX)

pc.create_index(
    name=INDEX,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


while True:
    desc = pc.describe_index(INDEX)
    if desc.get("status", {}).get("ready"):
        break
    time.sleep(0.5)

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = splitter.split_documents(document_list)
print("총 chunk 개수:", len(chunked_documents))

database = PineconeVectorStore.from_existing_index(
    index_name=INDEX,
    embedding=embedding,  
)

batch_size = 50 
for i in range(0, len(chunked_documents), batch_size):
    batch = chunked_documents[i:i+batch_size]
    database.add_documents(batch)
    print(f"업로드 진행: {i+1} ~ {i+len(batch)} / {len(chunked_documents)}")

In [15]:
query = '연봉 7천만원인 직장인의 소득세는 얼마인가요?'


In [16]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model=os.getenv("model"))

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [ ]:
retriever = database.as_retriever()
retriever.invoke(query)